## **Turi Create Image Classifier on Skafos**
The below example code and more documentation on the approach can be found [here](https://apple.github.io/turicreate/docs/userguide/image_classifier/).
This notebook does the following:
1. Loads the data
2. Builds the model
3. Saves the model

In [ ]:
import turicreate as tc
import urllib
import tarfile
import coremltools
import save_models as sm
from skafossdk import *

In [ ]:
ska  = Skafos() # initialize Skafos

### 1. **Load the data**
The data for this example are 25000 images, which breaks down to 12500 Cats and 12500 Dogs, and can be found [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765). The data is then split into training and testing datasets.

In [ ]:
data_url = "https://s3.amazonaws.com/skafos.example.data/PetImages.tar.gz"
data_path = "PetImages.tar.gz"

# download the data and extract
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
# load the data into SFrame, add a label column
data = tc.image_analysis.load_images('PetImages', with_path=True, ignore_failure = True)
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

# test train split
train_data, test_data = data.random_split(0.8)

### 2. **Build the model**
We pass the data to the `tc.image_classifer.create` function and specify a few arguments needed to properly run the model. To understand more about this, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/image_classifier/).


In [ ]:
# Create the model
model = tc.image_classifier.create(train_data, target='label')

### 3. **Save the model**
We save the model to Skafos so that you can download it into your project later without having to always retrain the model.

In [ ]:
# Save the model for later use in Turi Create
coreml_model_name = 'image_classifier.mlmodel'
res = model.export_coreml(coreml_model_name)

# use coremltools utility function to convert model weights to half-precision
model_spec = coremltools.utils.load_spec(coreml_model_name)
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
                                compressed_model = compressed_model,
                                permissions = 'public')